In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import esmtools as et
from esmtools import stats

import shapefile
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gp
import regionmask
import matplotlib.patches as mpatches
import xesmf as xe

In [2]:
# Open dataset 
dr = '/home/jovyan/SOARS_2021/Data/MRI-ESM2-0/'
fn = 'thetao_Omon_MRI-ESM2-0_omip2_r1i1p1f1_gr_195801-201812.nc'

data_thetao = xr.open_dataset(dr+fn)
data_thetao

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lev: 61, lon: 360, time: 732)
Coordinates:
  * time       (time) datetime64[ns] 1958-01-16T12:00:00 ... 2018-12-16T12:00:00
  * lev        (lev) float64 1.0 3.5 6.5 10.0 ... 5.5e+03 6.15e+03 6.525e+03
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lev_bnds   (lev, bnds) float64 ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    thetao     (time, lev, lat, lon) float32 ...
Attributes: (12/36)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           OMIP
    branch_method:         no parent
    creation_date:         2020-03-26T15:20:29Z
    data_specs_version:    01.00.31
    experiment:            OMIP experiment forced by JRA55-do atmospheric dat...
    ...                    ...
    title:                 MRI-ESM2-0 output prepared for CMIP6
    variable_id:           thetao
    variant_label:         r1i1p1f1
    license:               CMIP6 model data produced by MRI is licensed under...
    cmor_version:          3.5.0
    tracking_id:           hdl:21.14100/34bd754e-ec3c-4c8b-9acc-add54b9b79ac

In [3]:
#fn_area = xr.open_dataset('/home/jovyan/SOARS_2021/Data/CESM2/tarea_180x360_062221.nc')
# = fn_area.TAREA
#TAREA

In [4]:
# Change the time to be from DatetimeNoLeap to a Datetime64
data_thetao['time'] = np.arange('1958-01-01','2019-01-01',dtype = 'datetime64[M]')
data_thetao

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lev: 61, lon: 360, time: 732)
Coordinates:
  * time       (time) datetime64[ns] 1958-01-01 1958-02-01 ... 2018-12-01
  * lev        (lev) float64 1.0 3.5 6.5 10.0 ... 5.5e+03 6.15e+03 6.525e+03
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 1958-01-01 1958-02-01 ... 2019-01-01
    lev_bnds   (lev, bnds) float64 0.0 2.0 2.0 5.0 ... 6.5e+03 6.5e+03 6.55e+03
    lat_bnds   (lat, bnds) float64 -90.0 -89.0 -89.0 -88.0 ... 89.0 89.0 90.0
    lon_bnds   (lon, bnds) float64 0.0 1.0 1.0 2.0 ... 358.0 359.0 359.0 360.0
    thetao     (time, lev, lat, lon) float32 ...
Attributes: (12/36)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           OMIP
    branch_method:         no parent
    creation_date:         2020-03-26T15:20:29Z
    data_specs_version:    01.00.31
    experiment:            OMIP experiment forced by JRA55-do atmospheric dat...
    ...                    ...
    title:                 MRI-ESM2-0 output prepared for CMIP6
    variable_id:           thetao
    variant_label:         r1i1p1f1
    license:               CMIP6 model data produced by MRI is licensed under...
    cmor_version:          3.5.0
    tracking_id:           hdl:21.14100/34bd754e-ec3c-4c8b-9acc-add54b9b79ac

In [5]:
# select dimensions!
#data_thetao['time'] = pd.date_range("1958-01", "2018-12", freq="MS")
thetao = data_thetao['thetao']

In [6]:
blob = thetao.sel(lon=slice(210,225),lat=slice(40,50),time=slice('2013-07','2016-06'),lev=slice(0,500))
blob

<xarray.DataArray 'thetao' (time: 36, lev: 34, lat: 10, lon: 15)>
[183600 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2013-07-01 2013-08-01 ... 2016-06-01
  * lev      (lev) float64 1.0 3.5 6.5 10.0 15.0 ... 362.5 400.0 440.0 485.0
  * lat      (lat) float64 40.5 41.5 42.5 43.5 44.5 45.5 46.5 47.5 48.5 49.5
  * lon      (lon) float64 210.5 211.5 212.5 213.5 ... 221.5 222.5 223.5 224.5
Attributes:
    standard_name:  sea_water_potential_temperature
    long_name:      Sea Water Potential Temperature
    comment:        Diagnostic should be contributed even for models using co...
    units:          degC
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello volume: volcello
    history:        2020-03-26T15:20:28Z altered by CMOR: replaced missing va...

In [7]:
nonblob = thetao.sel(lon=slice(210,225),lat=slice(40,50),time=slice('1982-01','2010-12'),lev=slice(0,500))
nonblob

<xarray.DataArray 'thetao' (time: 348, lev: 34, lat: 10, lon: 15)>
[1774800 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1982-01-01 1982-02-01 ... 2010-12-01
  * lev      (lev) float64 1.0 3.5 6.5 10.0 15.0 ... 362.5 400.0 440.0 485.0
  * lat      (lat) float64 40.5 41.5 42.5 43.5 44.5 45.5 46.5 47.5 48.5 49.5
  * lon      (lon) float64 210.5 211.5 212.5 213.5 ... 221.5 222.5 223.5 224.5
Attributes:
    standard_name:  sea_water_potential_temperature
    long_name:      Sea Water Potential Temperature
    comment:        Diagnostic should be contributed even for models using co...
    units:          degC
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello volume: volcello
    history:        2020-03-26T15:20:28Z altered by CMOR: replaced missing va...